In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
total_bill,244.0,19.785943,8.902412,3.07,13.3475,17.795,24.1275,50.81
tip,244.0,2.998279,1.383638,1.00,2.0000,2.900,3.5625,10.00
size,244.0,2.569672,0.951100,1.00,2.0000,2.000,3.0000,6.00


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [9]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

### Tasks we don when we see a dataset:

1. Handing missing values 
2. Handling outliers
3. Feature scaling

### Automate enitre process.

In [11]:
categorical=[cols for cols in df.columns if df[cols].dtype=='category']
numeric=[cols for cols in df.columns if df[cols].dtypes!='category']

In [12]:
categorical

['sex', 'smoker', 'day', 'time']

In [13]:
numeric

['total_bill', 'tip', 'size']

In [19]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [22]:
df['time'].value_counts()

### Dinner - 0
### Lunch - 1

0    176
1     68
Name: time, dtype: int64

### Automate using pipeline

In [23]:
X=df.drop(['time'] , axis=1)

In [25]:
y=df['time']

In [27]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [28]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test , y_train, y_test =train_test_split(X , y , test_size=0.20 , random_state=42)

In [30]:
X_train

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2
...,...,...,...,...,...,...
106,20.49,4.06,Male,Yes,Sat,2
14,14.83,3.02,Female,No,Sun,2
92,5.75,1.00,Female,Yes,Fri,2
179,34.63,3.55,Male,Yes,Sun,2


## Build Pipleline for automating the regurlar tasks like handling missing data , Hadling categorical data , Scaling

In [31]:
from sklearn.impute import SimpleImputer ## Handle missing values
from sklearn.preprocessing import OneHotEncoder ## Handle nomial categorical features
from sklearn.preprocessing import StandardScaler ## For scaling the data 
from sklearn.pipeline import Pipeline ## Import pipleine
from sklearn.compose import ColumnTransformer

In [32]:
print(categorical)
print(numeric)

['sex', 'smoker', 'day', 'time']
['total_bill', 'tip', 'size']


In [33]:
categorical.remove('time')

In [35]:
print(categorical)
print(numeric)

['sex', 'smoker', 'day']
['total_bill', 'tip', 'size']


In [48]:
## Numeric Pipeline

num_pipeline = Pipeline(

    steps=[
        ('imputer' , SimpleImputer(strategy='median')) , ##Handling missing values
        ('scaler' , StandardScaler()) ## Scaling
    ]
)

In [49]:
##Categorical Pipeline

cat_pipeline=Pipeline(
    
    steps=[
        ('imputer' , SimpleImputer(strategy='most_frequent')) , ## Handle missing values 
        ('ohe' , OneHotEncoder()) ## Onehotencoder
    ]

)

In [50]:
##Combine both the pipeline
preprocessor=ColumnTransformer([
    ('num_pipeline' , num_pipeline , numeric),
    ('cat_pipeline' , cat_pipeline , categorical)
])

In [52]:
X_train =  preprocessor.fit_transform(X_train)

In [53]:
X_test = preprocessor.transform(X_test)

In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB , GaussianNB

## Automate model training

In [78]:
models={
    'Random Forest': RandomForestClassifier(),
    'Decision Tree' : DecisionTreeClassifier(),
    'SVC' : SVC() ,
    'Logistic Regression' : LogisticRegression() ,
    'Gaussain Naive Bayes' : GaussianNB() ,
    'Bernoulli Naive Bayes' : BernoulliNB()
}

In [79]:
from sklearn.metrics import accuracy_score

def evaluate_model(X_train , X_test , y_train , y_test , models):
    
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train , y_train)
        
        y_pred=model.predict(X_test)
        
        test_accuracy=accuracy_score(y_test , y_pred)
        
        report[list(models.keys())[i]] = test_accuracy
    return report

In [80]:
d={
    'Name' : 'Mohit' ,
    'Age': 22 , 
    'Gender' : 'Male'
}
list(d.values())[2]

'Male'

In [81]:
evaluate_model(X_train , X_test , y_train , y_test , models)

{'Random Forest': 0.9591836734693877,
 'Decision Tree': 0.9387755102040817,
 'SVC': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Gaussain Naive Bayes': 0.9591836734693877,
 'Bernoulli Naive Bayes': 0.9795918367346939}

## Once the best model is selected perform Hyperparametr tuning . Here selecting Random Forest as goal is implement ensemble technique

In [82]:
clf=RandomForestClassifier()

In [83]:
from sklearn.model_selection import GridSearchCV

In [84]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10] ,
    'oob_score' : [True , False]
}


In [85]:
gscv=GridSearchCV(clf , param_grid=param_grid , refit =True , cv=3 , verbose=3)

In [86]:
gscv.fit(X_train , y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV 1/3] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, oob_score=True;, score=0.954 total time=   0.1s
[CV 2/3] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, oob_score=True;, score=0.985 total time=   0.0s
[CV 3/3] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, oob_score=True;, score=0.954 total time=   0.0s
[CV 1/3] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, oob_score=False;, score=0.954 total time=   0.0s
[CV 2/3] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, oob_score=False;, score=0.985 total time=   0.0s
[CV 3/3] END criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50, oob_score=False;, score=0.954 total time=   0.0s
[CV 1/3] END criterion=gini,

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200],
                         'oob_score': [True, False]},
             verbose=3)

In [87]:
gscv.best_params_

{'criterion': 'gini',
 'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 200,
 'oob_score': True}

In [88]:
gscv.best_score_

0.9743589743589745

In [89]:
model=RandomForestClassifier(criterion='gini' , max_depth=None , min_samples_leaf=1 , min_samples_split=5 , n_estimators=200 , oob_score=True)

In [90]:
model.fit(X_train , y_train)

RandomForestClassifier(min_samples_split=5, n_estimators=200, oob_score=True)

In [91]:
y_pred=model.predict(X_test)

In [92]:
print(accuracy_score(y_test , y_pred))

0.9591836734693877
